<hr>
<div style="background-color: lightgray; padding: 20px; color: black;">
<div>
<img src="https://th.bing.com/th/id/R.3cd1c8dc996c5616cf6e65e20b6bf586?rik=09aaLyk4hfbBiQ&riu=http%3a%2f%2fcidics.uanl.mx%2fwp-content%2fuploads%2f2016%2f09%2fcimat.png&ehk=%2b0brgMUkA2BND22ixwLZheQrrOoYLO3o5cMRqsBOrlY%3d&risl=&pid=ImgRaw&r=0" style="float: right; margin-right: 30px;" width="200"/> 
<font size="5.5" color="8C3061"><b>Seq2seq con atención para predicción en series de tiempo </b></font> <br>
<font size="4.5" color="8C3061"><b>Aprendizaje de Máquina II - Tarea 2 </b></font> 
</div>
<div style="text-align: left">  <br>
Edison David Serrano Cárdenas. <br>
MSc en Matemáticas Aplicadas <br>
CIMAT - Sede Guanajuato <br>
</div>

</div>
<hr>


## <font color="8C3061" >**Cargar Librerías**</font> 

In [25]:
import numpy as np
import pandas as pd
import random 

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)  # Activar el modo de cuaderno
sns.set_theme()

import yfinance as yf

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary

from sklearn.metrics import mean_squared_error

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device is:",device)

device is: cuda


In [3]:
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(SEED)

## <font color="8C3061" >**Enunciado del problema**</font> 

Considere el valor histórico por hora de cripto-monedas de al menos 7 series, incluyendo el bitcoin, obtenga N series de al menos una longitud de T = 100. 

Luego entrene un modelo para dadas las series S[:T-t] prediga la parte final de cada serie S[T-t:]. Un valor típico es t=5.

Los datos descárgelos usando la librería yahoo-finance 

### <font color="8C3061" >**Objetivo**</font>

- Usar el modelo de Seq2Seq con atención usando el refuerzo del profesor para predicción de series de tiempo temporales.
- Realizar una predicción final de la moneda de BTC-USD (Bitcoin) 

<p align="center">
<img src="https://www.mheducation.es/media/wysiwyg/Spain/Newsletter/Post/22-eco-18-por-que-sube-y-baja-e--precio-de-las-acciones-en-bolsa-600.jpg" style="float: center; margin-right: 30px;" width="500"/> 
</p>


## <font color="8C3061" >**Descripción y Preprocesamiento de los Datos**</font> 

Las Criptomonedas elegidas para este proyecto son:

1. **BTC-USD (Bitcoin)**: Primera criptomoneda, utilizada como reserva de valor y transferencias, con suministro limitado.
   
2. **ETH-USD (Ethereum)**: Plataforma para contratos inteligentes y dApps, con Ether como moneda nativa.
   
3. **BNB-USD (Binance Coin)**: Criptomoneda del intercambio Binance, usada para tarifas y servicios dentro de su ecosistema.
   
4. **ADA-USD (Cardano)**: Plataforma blockchain enfocada en seguridad y escalabilidad, con ADA como moneda nativa.
   
5. **SOL-USD (Solana)**: Blockchain rápida y de bajo costo para dApps y contratos inteligentes, con SOL como moneda.
   
6. **XRP-USD (Ripple)**: Criptomoneda para pagos internacionales rápidos y económicos, con enfoque en instituciones financieras.
   
7. **DOGE-USD (Dogecoin)**: Criptomoneda popular basada en un meme, usada para microtransacciones y propinas.

Descarga de los historicos de cada criptomoneda usando la librería yfinance

In [4]:
# Lista de criptomonedas
cryptos = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'XRP-USD', 'ADA-USD', 'SOL-USD', 'DOGE-USD']

# Descargar datos por hora
def download_crypto_data(tickers, period='1mo', interval='1h'):
    data = {}
    for ticker in tickers:
        crypto_data = yf.download(ticker, period=period, interval=interval)
        data[ticker] = crypto_data[['Open', 'High', 'Low', 'Close', 'Volume']]
    return data

# Descargar los datos de las criptomonedas
crypto_data = download_crypto_data(cryptos)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Visualización de los Datos

In [5]:
def prepare_data_for_plotting(data):
    dfs = []
    for ticker, df in data.items():
        df['Ticker'] = ticker  # Add a column for the cryptocurrency ticker
        dfs.append(df[['Open', 'High', 'Low', 'Close', 'Volume', 'Ticker']])  # Select relevant columns
    # Concatenate all data into one DataFrame
    crypto_data_pd = pd.concat(dfs)
    return crypto_data_pd

# Prepare the data
crypto_data_pd = prepare_data_for_plotting(crypto_data)
crypto_data_pd = crypto_data_pd.reset_index()

**Volumen**

In [26]:
fig = px.line(crypto_data_pd, 
              x='Datetime', 
              y='Volume', 
              color='Ticker', 
              title='Precios de criptomonedas',
              labels={'Volume': 'Volumen', 'Ticker': 'Criptomoneda', 'Datetime': 'Tiempo (horas)'})
fig.show()

Se observa que cada cryptomoneda está a una diferente escala, por lo que se normalizarán los datos. Para ello, se normalizara cada una de las variables de cada cryptomoneda por separado.

In [7]:
# Función para normalizar los datos usando media y desviación estándar
def normalize_data_zscore(data):
    normalized_data = {}
    stats = {}  # Para almacenar la media y la desviación estándar de cada criptomoneda
    for ticker, df in data.items():
        means = df[['Open', 'High', 'Low', 'Close', 'Volume']].mean()
        stds = df[['Open', 'High', 'Low', 'Close', 'Volume']].std()
        
        stats[ticker] = {'mean': means, 'std': stds}
        
        df[['Open', 'High', 'Low', 'Close', 'Volume']] = (df[['Open', 'High', 'Low', 'Close', 'Volume']] - means) / stds
        normalized_data[ticker] = df
    return normalized_data, stats

normalized_crypto_data, crypto_stats = normalize_data_zscore(crypto_data)

In [8]:
class CryptoDataset(Dataset):
    def __init__(self, data, t=5):
        self.data = []
        self.train = []
        self.test = []
        self.t = t
        
        for ticker, df in data.items():
            series = df[['Open', 'High', 'Low', 'Close', 'Volume']].values
            l = len(series)
            train_size = int(0.8*l)
            
            
            for i in range(l - 100):
                input_seq = series[i:i+(100-t)]  # Últimos 100-t pasos como entrada
                target_seq = series[i+(100-t):i+100, 3]  # Solo el precio de cierre ('Close') como salida
                
                self.data.append((input_seq, target_seq))
                
                if (i<train_size):
                    self.train.append((input_seq, target_seq))
                else:
                    self.test.append((input_seq, target_seq))
                
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        return torch.FloatTensor(x), torch.FloatTensor(y).unsqueeze(1)  # y debe ser de la forma (batch_size, 1)

def denormalize_close_value(ticker, normalized_value, stats):
    mean_close = stats[ticker]['mean']['Close']
    std_close = stats[ticker]['std']['Close']
    return normalized_value * std_close + mean_close

Datos Preprocesados:

In [9]:
# Crear el dataset
seq2seq_dataset = CryptoDataset(normalized_crypto_data)

# Dividir en training y test
train_dataset, test_dataset = seq2seq_dataset.train, seq2seq_dataset.test

In [10]:
print("Número de ejemplos de entrenamiento:\t", len(train_dataset))
print("Número de ejemplos de test:\t\t", len(test_dataset))

Número de ejemplos de entrenamiento:	 4172
Número de ejemplos de test:		 343


In [11]:
# Crear DataLoaders
BATCH_SIZE = 32

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

## <font color="8C3061" >**Definición del Modelo**</font> 

### <font color="8C3061" >**Encoder**</font> 

In [12]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.LSTM(input_size=input_size, 
                           hidden_size=hidden_size, 
                           num_layers=num_layers, 
                           batch_first=True,
                           dropout = 0.35)
    
    def forward(self, x):
        outputs, (hidden, cell) = self.rnn(x)
        return outputs, hidden, cell

### <font color="8C3061" >**Atención**</font> 

In [13]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attention = nn.Linear(2 * hidden_size, 1)

    def forward(self, hidden, encoder_outputs):
        # Ajustar el tamaño de hidden para que tenga la forma (batch_size, seq_len, hidden_size)
        hidden = hidden[-1].unsqueeze(1)  # Solo la capa superior de los estados ocultos
        hidden = hidden.repeat(1, encoder_outputs.size(1), 1)  # (batch_size, seq_len, hidden_size)
        
        # Concatenar hidden con encoder_outputs
        energy = torch.cat((hidden, encoder_outputs), dim=2)  # (batch_size, seq_len, 2 * hidden_size)
        attention = self.attention(energy).squeeze(2)  # (batch_size, seq_len)
        
        return F.softmax(attention, dim=1)


### <font color="8C3061" >**Decoder**</font> 

In [14]:
class Decoder(nn.Module):
    def __init__(self, output_size, hidden_size, num_layers=1):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size=output_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=0.35)
        # Capa de salida
        self.fc_out = nn.Linear(hidden_size*2, output_size)
        self.attention = Attention(hidden_size)
        
    def forward(self, input_step, hidden, cell, encoder_outputs):
        output, (hidden, cell) = self.lstm(input_step, (hidden, cell))
        output = output.squeeze(1)
        
        attention_weights = self.attention(hidden, encoder_outputs)
        attention_weights = attention_weights.unsqueeze(1)
        
        context_vector = torch.bmm(attention_weights, encoder_outputs).squeeze(1)
        
        output = torch.cat((output, context_vector), 1)
        output = self.fc_out(output)
        
        return output, hidden, cell

### <font color="8C3061" >**Modelo Seq2Seq**</font> 

In [15]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, x, y=None, teacher_forcing_ratio=0.5):
        
        batch_size = x.size(0)
        seq_len = x.size(1)
        
        output_size = self.decoder.fc_out.out_features
        
        encoder_outputs, hidden, cell = self.encoder(x)
        outputs = torch.zeros(batch_size, 5, output_size).to(self.device)
        
        input_step = torch.zeros(batch_size, 1, output_size).to(self.device)
        
        for t in range(5):
            output, hidden, cell = self.decoder(input_step, hidden, cell, encoder_outputs)
            outputs[:, t, :] = output
            # Decidir si se usará el valor real como entrada para el siguiente paso
            use_teacher_forcing = True if torch.rand(1).item() < teacher_forcing_ratio else False
            if use_teacher_forcing and y is not None:
                input_step = y[:, t, :].unsqueeze(1)  # Usar el valor real
            else:
                input_step = output.unsqueeze(1)  # Usar la predicción generada
            
        return outputs

## <font color="8C3061" >**Entrenamiento del Modelo**</font> 

In [16]:
def train_seq2seq(model, dataloader, optimizer, criterion, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for x_batch, y_batch in dataloader:
            optimizer.zero_grad()
            
            # Convertir a torch.float32
            x_batch = x_batch.to(device).float()
            y_batch = y_batch.to(device).float()
            y_batch = y_batch.unsqueeze(-1)
            
            # Forward pass
            output = model(x_batch, y_batch, teacher_forcing_ratio=0.5)
            
            # Calcular la pérdida
            loss = criterion(output, y_batch)
            
            # Backward y optimización
            loss.backward()
            optimizer.step()
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

In [17]:
encoder = Encoder(input_size=5, hidden_size=64).to(device)
decoder = Decoder(output_size=1, hidden_size=64).to(device)
model = Seq2Seq(encoder, decoder, device).to(device)

# Optimizer y Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

/media/edserranoc/Data/EDSC_2024/Code/CIMAT/Machine_Learning2/mlenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:88: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.35 and num_layers=1



In [18]:
dataset_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
train_seq2seq(model, dataset_loader, optimizer, criterion, num_epochs=10)

Epoch 1/10, Loss: 0.07107477635145187
Epoch 2/10, Loss: 0.0666191354393959
Epoch 3/10, Loss: 0.03648287430405617
Epoch 4/10, Loss: 0.03264553099870682
Epoch 5/10, Loss: 0.05829419195652008
Epoch 6/10, Loss: 0.039682526141405106
Epoch 7/10, Loss: 0.04013684764504433
Epoch 8/10, Loss: 0.0760193020105362
Epoch 9/10, Loss: 0.030654750764369965
Epoch 10/10, Loss: 0.01463246438652277


Cálcular el error en el conjunto dedicado a testeo

In [19]:
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()  # Coloca el modelo en modo evaluación

true_close_values = []
predicted_close_values = []

with torch.no_grad():  # No calcules gradientes durante la evaluación
    for x_batch, y_batch in test_loader:
        x_batch = x_batch.to(device).float()
        y_batch = y_batch.to(device).float()
        y_batch = y_batch.unsqueeze(-1)
        
        output = model(x_batch)

        predicted_close_values.extend(output[:, :, 0].cpu().numpy())
        true_close_values.extend(y_batch[:, :, 0].cpu().numpy())

true_close_values = np.array(true_close_values)
predicted_close_values = np.array(predicted_close_values)

# Calcular el error cuadrático medio
mse = mean_squared_error(true_close_values, predicted_close_values)
print(f'MSE para el conjunto de test: {mse}')

MSE para el conjunto de test: 0.040845729410648346


## <font color="8C3061" >**Inferencia de la Criptomoneda BitCoin**</font> 

In [20]:
bitcoin_x = normalized_crypto_data['BTC-USD'][-100:-5][['Open', 'High', 'Low', 'Close', 'Volume']].values
bitcoin_y = normalized_crypto_data['BTC-USD'][-5:][['Close']].values
time_bitcoin = normalized_crypto_data['BTC-USD'][-100:].index

bitcoin_x = torch.FloatTensor(bitcoin_x).unsqueeze(0).to(device)
bitcoin_y = torch.FloatTensor(bitcoin_y).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    output = model(bitcoin_x)
    predicted_close_values = output.squeeze(0).cpu().numpy()
    true_close_values = bitcoin_y.squeeze(0).cpu().numpy()
    
predicted_close_values = denormalize_close_value('BTC-USD', predicted_close_values, crypto_stats)
true_close_values = denormalize_close_value('BTC-USD', normalized_crypto_data['BTC-USD']['Close'], crypto_stats)     

Después de desnornalizar los datos y graficarlos se puede observar que la predicción no es buena. Incluso tiene mejor desempeño si se considera que la serie permanece constante

In [27]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_bitcoin, y=true_close_values, mode='lines+markers', name='Serie de tiempo real'))
fig.add_trace(go.Scatter(x=time_bitcoin[-5:], y=predicted_close_values.T[0], mode='lines+markers', name='Predicción'))
fig.update_layout(title='Predicción de precios de cierre de Bitcoin',
                    xaxis_title='Tiempo',
                    yaxis_title='Precio de cierre')
fig.show()

## <font color="8C3061" >**Referencias**</font> 

La implementación de la arquitectura tomó a consideración varias de las ideas presentadas en los siguientes enlaces

[1] [Learning Pytorch Seq2Seq with M5 Data-Set - Kaggle](https://www.kaggle.com/code/omershect/learning-pytorch-seq2seq-with-m5-data-set) <br>
[2] [Implementing Seq2Seq Models for Efficient Time Series Forecasting - Medium](https://medium.com/@maxbrenner-ai/implementing-seq2seq-models-for-efficient-time-series-forecasting-88dba1d66187)